In [1]:
!pip install -q openai-whisper unsloth transformers torchaudio soundfile librosa datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 45.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 133.7 

In [ ]:
from huggingface_hub import login
login(token="hf_AkUDUmFkG")  # Replace with your token

In [3]:
import whisper

# Load Whisper model (small for efficiency; use "medium" for better accuracy if needed)
whisper_model = whisper.load_model("small").to("cuda")  # GPU for speed

print("Whisper loaded! Handles audio encoding/decoding automatically.")

100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 228MiB/s]


Whisper loaded! Handles audio encoding/decoding automatically.


In [4]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Qwen2.5-3B-bnb-4bit"  # Fine-tuned for reasoning/QA

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True  # Dynamic 4-bit: Preserves precision for critical params
)

FastLanguageModel.for_inference(model)  # Faster generation

print("Quantized model loaded! VRAM efficient (~4-6GB).")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Quantized model loaded! VRAM efficient (~4-6GB).


In [27]:

audio_path = "/content/Recording.m4a"  # Or your uploaded file name

print("Sample audio ready! For custom: Replace with audio asking a question like 'What is AI?'")

Sample audio ready! For custom: Replace with audio asking a question like 'What is AI?'


In [28]:
import torchaudio
import soundfile as sf
import librosa

def transcribe_audio(audio_paths, batch_size=1):
    """
    Transcribe one or more audios efficiently.
    - Handles encoding: Resamples to 16kHz if needed.
    - Batching: Process in batches to save memory.
    - GPU: Whisper uses GPU for transcription.
    """
    if isinstance(audio_paths, str):
        audio_paths = [audio_paths]  # Single file to list

    transcriptions = []
    for i in range(0, len(audio_paths), batch_size):
        batch = audio_paths[i:i + batch_size]
        for path in batch:
            # Load and preprocess audio (resample to 16kHz for Whisper)
            audio, sr = librosa.load(path, sr=16000)  # Efficient resampling
            result = whisper_model.transcribe(audio)
            text = result["text"].strip()
            transcriptions.append(text)
            print(f"Transcription for {path}: {text}")

    return transcriptions if len(transcriptions) > 1 else transcriptions[0]

print("Transcription function ready! Handles batching for multiple audios.")

Transcription function ready! Handles batching for multiple audios.


In [29]:
def reason_on_text(text, max_tokens=400):
    """
    Clean, duplication-free reasoning with deeper technical detail.
    """
    prompt = f"""You are an expert AI teacher. Explain the query in depth with step-by-step logical reasoning.
Include technical details where relevant (e.g., for neural networks: input/hidden/output layers, weights, activation functions, backpropagation).

Query: {text}

Provide a complete, structured explanation:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.6,             # Lower for more consistent/factual
        do_sample=True,
        repetition_penalty=1.5,      # Even stronger anti-repetition
        eos_token_id=tokenizer.eos_token_id,
    )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Robust split: Remove prompt reliably
    if prompt in full_text:
        answer = full_text.split(prompt, 1)[-1].strip()
    else:
        answer = full_text[len(prompt):].strip()  # Fallback

    # Clean escaped newlines and extra whitespace
    answer = answer.replace("\\n\\n", "\n\n").replace("\\n", "\n").replace("\n\n\n", "\n\n")

    return answer

In [30]:
def speech_to_reasoning(audio_path):
    print("=== Starting Speech-to-Reasoning Pipeline ===\n")

    # Step 1: Transcribe
    print("Step 1: Transcribing audio with Whisper...")
    transcription = transcribe_audio(audio_path)

    # Step 2: Reasoning
    print("\nStep 2: Performing logical reasoning with quantized LLM...")
    response = reason_on_text(transcription)

    # VRAM Check
    print("\n=== VRAM Usage (Post-Pipeline) ===")
    !nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

    # Final Clean Output
    print("\n" + "="*60)
    print("FINAL PIPELINE RESULT")
    print("="*60)
    print(f"Input Audio: {audio_path}")
    print(f"Transcription:\n{transcription}\n")
    print(f"Reasoned Response:\n{response}")
    print("="*60)

    return response

# Run the demo
speech_to_reasoning(audio_path)

=== Starting Speech-to-Reasoning Pipeline ===

Step 1: Transcribing audio with Whisper...


/tmp/ipython-input-741106211.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(path, sr=16000)  # Efficient resampling


Transcription for /content/Recording.m4a: What is machine learning? Explain how your network works.

Step 2: Performing logical reasoning with quantized LLM...

=== VRAM Usage (Post-Pipeline) ===
4472, 15360

FINAL PIPELINE RESULT
Input Audio: /content/Recording.m4a
Transcription:
What is machine learning? Explain how your network works.

Reasoned Response:
introduction + 3 main points of analysis = conclusion

**Introduction**

Machine Learning refers to algorithms that enable computers and software systems learn from data inputs without being explicitly programmed by human programmers or engineers using traditional statistical methods like regression models; however they don't rely solely on rules-based approaches but rather adapt themselves based upon experience through training processes designed specifically tailored towards particular tasks such as classification problems - it allows machines "learn" patterns within their environment which could then be applied more broadly acros

'introduction + 3 main points of analysis = conclusion\n\n**Introduction**\n\nMachine Learning refers to algorithms that enable computers and software systems learn from data inputs without being explicitly programmed by human programmers or engineers using traditional statistical methods like regression models; however they don\'t rely solely on rules-based approaches but rather adapt themselves based upon experience through training processes designed specifically tailored towards particular tasks such as classification problems - it allows machines "learn" patterns within their environment which could then be applied more broadly across other similar scenarios i.e predicting future events etc..\n\nIn this context we will focus our discussion around Neural Networks because its one type among many types available today including Decision Trees & Support Vector Machines(SVM). \n\nNeural Network consists mainly three components namely Input Layer / Hidden Layers(Recurrent/LSTM) , Output

In [13]:
# Monitor GPU memory (ensure optimal use)
!nvidia-smi

import torch
print(f"VRAM Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

Fri Jan  9 09:35:52 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P0             28W /   70W |    4470MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----